<a href="https://colab.research.google.com/github/itspalash/NLP/blob/main/Refrence_Trail_Bert_Exp_CPC_Codes_(via_classification_and_regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!pip install datasets transformers
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.4.0
!pip install tensorflow-text

     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 4.0 MB 35.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 212 kB 46.4 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 136 kB 51.8 MB/s 
     |████████████████████████████████| 127 kB 52.9 MB/s 
     |████████████████████████████████| 6.6 MB 41.6 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 48.3 MB/s 
     |████████████████████████████████| 271 kB 50.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [ ]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c us-patent-phrase-to-phrase-matching

  0% 0.00/682k [00:00<?, ?B/s]
100% 682k/682k [00:00<00:00, 116MB/s]


In [ ]:
! unzip us-patent-phrase-to-phrase-matching

Archive:  us-patent-phrase-to-phrase-matching.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks


In [ ]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Set up tokenizer to generate Tensorflow dataset
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

print("Vocab size:", len(tokenizer.vocab))

Vocab size: 30522


In [ ]:
tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])

[101, 102]

In [ ]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s.numpy()))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
train_csv_tf = tf.data.experimental.make_csv_dataset("train.csv", batch_size=100)

In [ ]:
import pandas as pd

train_pd_np = pd.read_csv('train.csv')
test_pd_np = pd.read_csv('train.csv')
cpc_code = pd.read_csv('cpc_code.csv')


In [ ]:
#len(train_pd_np.context.unique())
train_final = pd.merge(train_pd_np, cpc_code, on="context")
test_final = pd.merge(test_pd_np, cpc_code, on="context")
test_final.head()


,id,anchor,target,context,score,title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...


In [ ]:
data = {"anchor": tf.ragged.constant(train_final['anchor'].to_numpy()),
"target": tf.ragged.constant(train_final['target'].to_numpy()),
"context": tf.ragged.constant(train_final['title'].to_numpy()),
"score": tf.ragged.constant(train_final['score'].to_numpy())}

data_test = {"anchor": tf.ragged.constant(test_final['anchor'].to_numpy()),
"target": tf.ragged.constant(test_final['target'].to_numpy()),
"context": tf.ragged.constant(test_final['title'].to_numpy()),
"score": tf.ragged.constant(test_final['score'].to_numpy())}

In [ ]:
def bert_encode(glue_dict, tokenizer):


  anchor = tf.ragged.constant([encode_sentence(s) for s in data["anchor"]])
  target = tf.ragged.constant([encode_sentence(s) for s in data["target"]])
  context = tf.ragged.constant([encode_sentence(s) for s in data["context"]])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*anchor.shape[0]
  input_word_ids = tf.concat([cls, anchor, target, context], axis=-1)
  input_mask = tf.ones_like(input_word_ids).to_tensor()

  type_cls = tf.zeros_like(cls)
  type_target = tf.zeros_like(target)
  type_anchor = tf.ones_like(anchor)
  type_context = tf.ones_like(context)
  input_type_ids = tf.concat([type_cls, type_anchor, type_target,type_context], axis=-1).to_tensor()

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [ ]:
data_train = bert_encode(data, tokenizer)
data_train_labels = data["score"] 

In [ ]:
data_test

{'input_mask': <tf.Tensor: shape=(36473, 75), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(36473, 75), dtype=int32, numpy=
 array([[0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        ...,
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(36473, 75), dtype=int32, numpy=
 array([[  101, 19557, 18532, ...,     0,     0,     0],
        [  101, 19557, 18532, ...,     0,     0,     0],
        [  101, 19557, 18532, ...,     0,     0,     0],
        ...,
        [  101,  4773,  4651, ...,     0,     0,     0],
        [  101,  4773,  4651, ...,     0,     0,     0],
        [  101,  4773,  4651, ...,     0, 

In [ ]:
data_Test = bert_encode(data_test, tokenizer)
data_Test_labels = data_test["score"] 

In [ ]:
data_train

{'input_mask': <tf.Tensor: shape=(36473, 75), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(36473, 75), dtype=int32, numpy=
 array([[0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        ...,
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(36473, 75), dtype=int32, numpy=
 array([[  101, 19557, 18532, ...,     0,     0,     0],
        [  101, 19557, 18532, ...,     0,     0,     0],
        [  101, 19557, 18532, ...,     0,     0,     0],
        ...,
        [  101,  4773,  4651, ...,     0,     0,     0],
        [  101,  4773,  4651, ...,     0,     0,     0],
        [  101,  4773,  4651, ...,     0, 

In [ ]:
for key, value in data_train.items():
  print(f'{key:15s} shape: {value.shape}')

print(f'glue_train_labels shape: {data_train_labels.shape}')

input_word_ids  shape: (36473, 75)
input_mask      shape: (36473, 75)
input_type_ids  shape: (36473, 75)
glue_train_labels shape: (36473,)


In [ ]:
import tensorflow_text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
preprocessed_text1 = bert_encode(data, tokenizer)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
bert_preprocess(["text_input"])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
bert_encoder(bert_preprocess(["text_input"]))

[<tf.Tensor: shape=(1, 1, 768), dtype=float32, numpy=
 array([[[-3.70505303e-01,  3.48789155e-01,  1.36822259e+00,
          -4.12046194e-01, -1.41938591e+00,  6.82782114e-01,
          -4.01294678e-01,  7.82847106e-01, -1.49219728e+00,
           1.31333709e+00, -7.67044485e-01, -1.44322775e-02,
          -9.87113416e-01, -5.26317775e-01, -9.83143926e-01,
           9.59602416e-01, -1.76328075e+00, -6.59584045e-01,
          -1.28117597e+00, -4.30466473e-01, -8.24903071e-01,
          -1.23323095e+00,  1.92763782e+00,  1.41038966e+00,
           2.98254609e-01, -1.82035089e+00, -1.97779751e+00,
          -6.12401724e-01, -2.23916483e+00, -5.17469525e-01,
          -6.24932587e-01,  3.96317849e-03, -1.45558083e+00,
          -8.30678791e-02, -4.50654536e-01,  1.27843261e+00,
           3.57814394e-02, -1.17306650e+00,  3.48038107e-01,
           1.36520863e+00, -2.21299982e+00,  2.36031413e-01,
          -1.42809236e+00,  9.50261280e-02,  5.29212952e-02,
           5.90637684e-01,  3.3

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

MODEL

In [ ]:
import json

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [ ]:
bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=1)

In [ ]:
bert_classifier.summary()

Model: "bert_classifier"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 bert_encoder_1 (BertEncoder)   [(None, None, 768),  109482240   ['input_word_ids[0][0]',         
                                 (None, 768)]                     'input_mask[0][0]'

In [ ]:
glue_batch = {key: val[:10] for key, val in data_train.items()}

results = bert_classifier(
    glue_batch, training=False
).numpy()
results

array([[0.32531622],
       [0.32915953],
       [0.33468887],
       [0.33347988],
       [0.33715686],
       [0.3351959 ],
       [0.34196556],
       [0.3285494 ],
       [0.32908943],
       [0.3352259 ]], dtype=float32)

In [ ]:
print(results[0])
#print(tf.nn.softmax(results[0]))
#print(tf.argmax(tf.nn.softmax(results[0])))

[-0.10645445]


In [ ]:
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)
checkpoint.read(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()

In [ ]:
epochs = 2
batch_size = 32
eval_batch_size = 32

train_data_size = len(data_train_labels)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

In [ ]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

rmetrics = tf.keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=tf.float32)
"""
bert_classifier.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)
"""
bert_classifier.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=rmetrics)
     
bert_classifier.fit(
      data_train, data_train_labels,
      batch_size=64,
      epochs=epochs)

Epoch 1/2
570/570 [==============================] - 1000s 2s/step - loss: 0.2335 - mean_squared_error: 0.0837
Epoch 2/2
499/570 [=========================>....] - ETA: 2:01 - loss: 0.2252 - mean_squared_error: 0.0766

KeyboardInterrupt: ignored

In [ ]:
results = bert_classifier.evaluate(data_Test, data_Test_labels, batch_size=64)

570/570 [==============================] - 350s 613ms/step - loss: 0.2165 - mean_squared_error: 0.0708


In [ ]:
print("test loss, test acc:", results)

test loss, test acc: [0.21647381782531738, 0.07080849260091782]


In [ ]:
glue_batch = {key: val[:10] for key, val in data_Test.items()}

bert_classifier(
    glue_batch, training=False
).numpy()

array([[0.2982409 ],
       [0.2982409 ],
       [0.29824087],
       [0.2982409 ],
       [0.29824084],
       [0.2982409 ],
       [0.2982409 ],
       [0.2982409 ],
       [0.2982409 ],
       [0.29824093]], dtype=float32)

In [ ]:
bert_classifier.save_weights('checkpoints/my_checkpoint')

In [ ]:
new_bert_classifier, new_bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=1)

# Restore the weights
new_bert_classifier.load_weights('checkpoints/my_checkpoint')

In [ ]:
import shutil
shutil.make_archive("checkpoints_zip", 'zip', "checkpoints")

'/content/checkpoints_zip.zip'

In [ ]:
# Classifier 
result = bert_classifier(glue_batch, training=False)

result = tf.argmax(result).numpy()


In [ ]:
result

array([9, 1, 9, 9, 3])